# Introduction

This is my first analysis publishing in kaggle, I hope that you like and let me a feed back about anything including grammar.

I chose to compare some Machine Learning models who usually works as decision tree classifiers. The final chose model it's that have the best hit percentage, after that the model will be use to answer the test data set.

This kernel has four main sections; Null data, Extracting Features, Training data analysis and Model. If you have any idea that might improve this kernel, please be sure to comment, or fork and experiment as you like. If you didn't understand any part, feel free to ask.

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

    

In [ ]:
# Import data
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data_final = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
# Looking until fifth row of dataframe
train_data.head()

In [ ]:
test_data_final.head()

## Information Provided by the Kaggle

* PassengerId is the unique id of the row and it doesn't have any effect on target
* Survived is the target variable we are trying to predict (0 or 1):
 * 1 = Survived
 * 0 = Not Survived
* Pclass (Passenger Class) is the socio-economic status of the passenger and it is a categorical ordinal feature which has 3 unique values (1, 2 or 3):
 * 1 = Upper Class
 * 2 = Middle Class
 * 3 = Lower Class
* Name, Sex and Age
* SibSp is the total number of the passengers' siblings and spouse
* Parch is the total number of the passengers' parents and children
* Ticket is the ticket number of the passenger
* Fare is the passenger fare
* Cabin is the cabin number of the passenger
* Embarked is port of embarkation and it is a categorical feature which has 3 unique values (C, Q or S):
 * C = Cherbourg
 * Q = Queenstown
 * S = Southampton

In [ ]:
train_data.info()

In [ ]:
test_data_final.info()

Null values in both the training and test data, in both the Cabin and Age column. Thus, the Cabin column of this analysis was discarded, for the Age column the missing values were replaced, based on the average age of the passengers considering the Pclass and Sex.

# Null data

In [ ]:
null_train = train_data.isnull().sum()
null_test = test_data_final.isnull().sum()
pd.DataFrame([null_train, null_test],index=['Train_Data', 'Test_Data'])

Null data at columns Age, Fare e Embarked. The Test_Data's Survived column, because it isn't have this column.

## Fare

In [ ]:
test_data_final[test_data_final['Fare'].isnull()]

In [ ]:
# finding a fare value for Storey, Mr. Thomas.
Fare_Mr_Thomas = train_data.query('Pclass == 3 & Embarked == "S" & Sex == "male" & Age >= 60').Fare.mean()
Fare_Mr_Thomas

In [ ]:
test_data_final['Fare'].fillna(Fare_Mr_Thomas, inplace = True)
test_data_final[test_data_final['Fare'].isnull()].sum()

## Embarked

In [ ]:
train_data[train_data['Embarked'].isnull()] 

In [ ]:
train_data.query('Pclass == 1  & Fare > 70  & Fare < 90 ')['Embarked'].unique()

Missing values in Embarked are filled with S with this information.

> "When I googled Stone, Mrs. George Nelson (Martha Evelyn), I found that she embarked from S (Southampton) with her maid Amelie Icard, in this page Martha Evelyn Stone: Titanic Survivor."

> "Mrs Stone boarded the Titanic in Southampton on 10 April 1912 and was travelling in first class with her maid Amelie Icard. She occupied cabin B-28."

**Reference**: https://www.kaggle.com/gunesevitan/titanic-advanced-feature-engineering-tutorial/notebook


---

In [ ]:
# Substuctin the values of Miss. Amelie ans Mrs. Gerorge
train_data['Embarked'].fillna('S', inplace = True)
train_data[train_data['Embarked'].isnull()].sum()

## Age

In [ ]:
age_null = train_data[train_data['Age'].isnull()]
age_null.sample(5)

In [ ]:
age_null.groupby(['Pclass','Embarked', 'Sex']).Sex.count()

In [ ]:
# adopted values
colunaum = train_data.groupby(['Sex', 'Pclass']).Age.mean().round()
colunaum

correcting dataframe ages 

In [ ]:
conveter = colunaum.to_dict()

In [ ]:
# Train_Data
coluna_final = []

for index, row in train_data.iterrows():
  Sex = row['Sex']
  pclass = row['Pclass']
  Age_nan = np.isnan(row['Age'])
  Age_table = row['Age']
  if (Age_nan == True):
    objeto_valo = conveter[Sex, pclass]
    coluna = [Age_nan, Sex, pclass, objeto_valo]
    coluna_final.append(coluna)
  else:
    coluna = [Age_nan, Sex, pclass, Age_table]
    coluna_final.append(coluna)


Corret_age = pd.DataFrame(coluna_final, columns = ['Age_cond', 'Sex', 'Pclass', 'key'])
Corret_age

In [ ]:
# replacing the values
train_data['Age'] = Corret_age.key

In [ ]:
# Test_data
coluna_final = []

for index, row in test_data_final.iterrows():
  Sex = row['Sex']
  pclass = row['Pclass']
  Age_nan = np.isnan(row['Age'])
  Age_table = row['Age']
  if (Age_nan == True):
    objeto_valo = conveter[Sex, pclass]
    coluna = [Age_nan, Sex, pclass, objeto_valo]
    coluna_final.append(coluna)
  else:
    coluna = [Age_nan, Sex, pclass, Age_table]
    coluna_final.append(coluna)


Corret_age = pd.DataFrame(coluna_final, columns = ['Age_cond', 'Sex', 'Pclass', 'key'])
Corret_age

In [ ]:
# replacing the values
test_data_final['Age'] = Corret_age.key

# Extracting Features

The treatment pronouns contained in the data, in the Name column, provide important information. In this way the feature class was added to the data.

In [ ]:
pronoun_train = train_data['Name']
pronoun_train = pronoun_train.str.split('(', expand=True)[0]
pronoun_test = test_data_final['Name'].str.split('(', expand=True)[0]

In [ ]:
lista_de_nomes = ['Mr','Mrs','Miss','Ms','Sir','Lady','Dr','Mme','Mlle','Countess','Col','Major','Capt','Don','Rev','Master','Jonkheer']
teste = pronoun_train.str[0:].str.extractall(r"(?P<pronome>\w+)")
teste = teste.rename(columns={'pronome': 'nomes'})
teste = teste.value_counts()
teste.loc[lista_de_nomes]

In [ ]:
contar = teste.loc[['Mr','Mrs','Miss','Ms','Sir','Lady','Dr','Mme','Mlle','Countess','Col','Major','Capt','Don','Rev','Master','Jonkheer']].sum()
contar

Based on the quantitative above, it was considered the classification into three categories:
* Mr - all male pronouns
* Miss - All feminine pronouns except Countess
* Dr.\Sir.\Capt.\Rev. - All other treatment pronouns found

In [ ]:
female_pronouns = ['Mrs','Miss','Ms','Lady','Mme','Mlle']
female_pronouns_test_data = ['Mrs','Miss','Ms','Lady','Mme','Mlle','Dona']
pronoun_other_categories = ['Sir','Dr','Col','Major','Capt','Don','Rev','Jonkheer', 'Countess', 'Master', 'the Countess']
# train_data['Name'].str.replace(('Mrs','Miss','Ms','Lady','Mme','Mlle','Countess'), 'Miss')

train_data['classes'] = pronoun_train.str[0:].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
test_data_final['classes'] = pronoun_test.str[0:].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
train_data.sample(10)

In [ ]:
train_data['classes'].replace(female_pronouns, 'Miss',inplace=True)
train_data['classes'].replace(pronoun_other_categories, 'Dr.\Sir.\Capt.\Rev.',inplace=True)
test_data_final['classes'].replace(female_pronouns_test_data, 'Miss',inplace=True)
test_data_final['classes'].replace(pronoun_other_categories, 'Dr.\Sir.\Capt.\Rev.',inplace=True)

In [ ]:
train_data.classes.value_counts()

In [ ]:
test_data_final.sample(10)

## Families

In [ ]:
testando = pd.concat([train_data['Name'].str[0:].str.split(', ', expand=True)[0], test_data_final['Name'].str[0:].str.split(', ', expand=True)[0]])
totais = pd.Series(testando.value_counts())
totais = totais.to_dict()

In [ ]:
lista_da_familia = []
for index, row in train_data.iterrows():
  nome = row['Name'].split(',')[0]
  if (totais[nome] <= 1):
    lista_da_familia.append('Alone')
  elif (totais[nome] <= 3):
    lista_da_familia.append('Small')
  elif (totais[nome] <= 6):
    lista_da_familia.append('Medium')
  else:
    lista_da_familia.append('Large')
  
train_data['Family_size'] = lista_da_familia
train_data

In [ ]:
lista_da_familia = []
for index, row in test_data_final.iterrows():
  nome = row['Name'].split(',')[0]
  if (totais[nome] <= 1):
    lista_da_familia.append('Alone')
  elif (totais[nome] <= 3):
    lista_da_familia.append('Small')
  elif (totais[nome] <= 6):
    lista_da_familia.append('Medium')
  else:
    lista_da_familia.append('Large')
  
test_data_final['Family_size'] = lista_da_familia
test_data_final

In [ ]:
train_data[['Family_size', 'Survived']].value_counts()

# Training data analysis

In [ ]:
train_data.describe()

In [ ]:
corr = train_data.corr()
corr

In [ ]:
import seaborn as sns
ax = sns.pairplot(train_data, y_vars="Survived", x_vars=["Pclass", 'Age', 'SibSp', 'Parch', 'Fare'],kind='reg', diag_kind=None)
ax.fig.suptitle('Dispersão entre as Varíaveis', fontsize=20, y=1.05)
ax

In [ ]:
import seaborn as sns
ax = sns.heatmap(corr)
ax

Based on the table and correlation graphs, one realizes that there is no linear relationship considerably between features. Emphasizing only that Pclass and Fare have a significant and expected negative correlation, since the price increases and the class decreases and if the price decreases the class increases, inversely proportional. Another relationship is between Parch and Sibsp, which has to do with the degree of kinship, showing a positive correlation.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

family_freq = testando.value_counts().value_counts() 

legenda = train_data.Family_size.value_counts().index


fig2 = plt.figure(constrained_layout=True, figsize=(10,16))

spec2 = gridspec.GridSpec(ncols=1, nrows=3, figure=fig2)
f2_ax1 = fig2.add_subplot(spec2[1, 0])
f2_ax2 = fig2.add_subplot(spec2[0, 0])
f2_ax3 = fig2.add_subplot(spec2[2, 0])


f2_ax1.bar(legenda, train_data.Family_size.value_counts())
f2_ax1.set_title('Total people by category', fontsize=25, pad=15)
f2_ax1.set_ylabel('Total', fontsize=15)
f2_ax1.tick_params(labelsize=15)

# Frequancia de pessoas por familia
f2_ax2.bar(family_freq.index.values, family_freq.values)
f2_ax2.set_xticks(range(0,9))
f2_ax2.set_xticklabels(family_freq.index.values)
f2_ax2.set_title('Frequency of people per family', fontsize=25, pad=15)
f2_ax2.set_xlabel('Number of people in the same family', fontsize=15)
f2_ax2.set_ylabel('Frequency', fontsize=15)
f2_ax2.tick_params(labelsize=15)

f2_ax3.bar(range(0,8,2), train_data[['Family_size', 'Survived']].value_counts()[:,0],color='red', label='C1',align= 'edge')
f2_ax3.bar(range(1,8,2), train_data[['Family_size', 'Survived']].value_counts()[:,1], color='green', label='C2',align= 'edge')
f2_ax3.set_xticks(range(1,8,2))
f2_ax3.set_xticklabels(legenda)
f2_ax3.legend(['Not Survived', 'Survived'], fontsize=20,loc=0)
f2_ax3.set_title('Survivors by family classification', fontsize=25, pad=15)
f2_ax3.set_ylabel('Total', fontsize=15)
f2_ax3.tick_params(labelsize=15)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5,4))
eixo = fig.add_axes([0,0,1,1])

mu, sigma = train_data['Age'].mean(), train_data['Age'].std()

eixo.hist(train_data['Age'], bins=80)
eixo.set_title('Histogram - Age Training Data', fontsize=15, pad=10)
eixo.set_xlabel('Age', fontsize=15)
eixo.set_ylabel('Frequency', fontsize=15)
eixo.grid(True)

eixo.annotate('$\mu = {0:.2f}$\n$\sigma = {1:.2f}$'.format(mu, sigma),
               xy=(50, 10), fontsize=20)

eixo.axvline(mu, color = 'k', linestyle='--')
eixo.annotate('mean', xy=(mu-1.3, 44), fontsize=20)

eixo.axvline(train_data['Age'].median(), color='g', linestyle='--')
eixo.annotate('average', xy=(train_data['Age'].median(), 35), fontsize=20)

In [ ]:
import seaborn as sns

sobreviveram = train_data[train_data.Survived == 1]
morreram = train_data[train_data.Survived == 0]
cores = {'Survived': 'g','Die': 'r', 'All': 'b'}

ax = sns.kdeplot(sobreviveram.Age, color='g')
ax = sns.kdeplot(morreram.Age, color='r')
ax = sns.kdeplot(train_data.Age, color='b')

ax.figure.set_size_inches(12, 6)
ax.set_title('Frequency Distribution - Age - KDE', fontsize=18)
ax.set_xlabel('Anos', fontsize=14)
ax.legend(cores, fontsize=20)
ax

In [ ]:
train_data.Sex.bool
ax = sns.boxplot( x = 'Age', y = 'Survived', data = train_data , orient = 'h')
ax.figure.set_size_inches(12, 4)
ax.set_title('Survived x Age', fontsize=18)
ax.set_xlabel('Years', fontsize=14)
ax.set_yticklabels(['Not Survived','Survived'])
ax

In [ ]:
sobreviveram = train_data[train_data.Survived == 1]
morreram = train_data[train_data.Survived == 0]
cores = {'Survived': 'g','Die': 'r', 'All': 'b'}

ax = sns.kdeplot(sobreviveram.Pclass, color='g')
ax = sns.kdeplot(morreram.Pclass, color='r')
ax = sns.kdeplot(train_data.Pclass, color='b')

ax.figure.set_size_inches(12, 6)
ax.set_title('Frequency Distribution - Pclass - KDE', fontsize=18)
ax.set_xlabel('Pclass', fontsize=14)
ax.legend(cores, fontsize=20)
ax

In [ ]:
fig = plt.figure(figsize=(5,4))
eixo = fig.add_axes([0,0,1,1])

eixo.pie(train_data.groupby('Sex').PassengerId.count(), labels=['man', 'women'],
         autopct='%.1f%%',explode=[0.1,0], shadow=True)
eixo.set_title('Percentage of passengers by sex', size=15, pad=10)

In [ ]:
fig = plt.figure(figsize=(5,4))
eixo = fig.add_axes([0,0,1,1])
eixo.pie(train_data.groupby(['Sex', 'Survived']).Survived.count(),labels=['Not Survived W.', 'Survived W.','Not Survived M.','Survived M.'],
         autopct='%.1f%%',explode=[0.1,0.1,0,0], shadow=True)
eixo.set_title('Total survivors and deaths by sex', size=15, pad=10)

In [ ]:
train_data.groupby(['Sex', 'Survived']).Survived.count()

In [ ]:
women = train_data.query('Sex == "female"')
women_survived = len(women.query('Survived == 1'))
women_survived_percent = (women_survived)/(women.shape[0])
print('Total of women: {}, total of survived women: {:.4%}({})'.format(women.shape[0],
                                                                   women_survived_percent,
                                                                   women_survived))

Survive = {0: 'No',
           1: 'Yes'}

Class = {1: '1º - Class',
         2: '2º - Class',
         3: '3º - Class'}
df_women_percent = pd.crosstab(women.Survived,
                               women.Pclass,
                               normalize = True) * 100
df_women_percent.rename(index = Survive, inplace = True)
df_women_percent.rename(columns = Class, inplace = True)
df_women_percent

In [ ]:
women.groupby(['Pclass','Embarked']).Age.describe()

In [ ]:
men = train_data.query('Sex == "male"')
men_survived = len(men.query('Survived == 1'))
men_survived_percent = (men_survived)/(men.shape[0])
print('Total of men: {}, total of survived men: {:.4%}({})'.format(men.shape[0],
                                                                   men_survived_percent,
                                                                   men_survived))


df_men_percent = pd.crosstab(men.Survived,
                               men.Pclass,
                               normalize = True) * 100
df_men_percent.rename(index = Survive, inplace = True)
df_men_percent.rename(columns = Class, inplace = True)
df_men_percent

In [ ]:
import plotly.graph_objects as go
import numpy as np
Series_1 = train_data.query('Survived == 1')
Series_2 = train_data.query('Survived == 0')
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=Series_1['PassengerId'], 
                           y=Series_1['Age'], z=Series_1['Pclass'],name='Survived', 
                           mode='markers', 
                           marker=dict(size=5,
                           color='green',                # set color to an array/list of desired values
                           colorscale='Viridis',   # choose a colorscale
                           opacity=0.8),
                           text= 'agrupador'))

fig.add_trace(go.Scatter3d(x=Series_2['PassengerId'], 
                           y=Series_2['Age'], z=Series_2['Pclass'],name='Not Survived', 
                           mode='markers', 
                           marker=dict(size=5,
                           color='red',                # set color to an array/list of desired values
                           colorscale='Viridis',   # choose a colorscale
                           opacity=0.8),
                           text= 'agrupador'))

fig.update_layout(scene=dict(xaxis_title='PassengerId',
                             yaxis_title='Age', zaxis_title='Pclass'))

fig.update_layout(scene = dict(
                    zaxis = dict(
                        ticktext= ['1º - Class','2º - Class','3º - Class'],
                        tickvals= [1, 2, 3]))                    
                  )


fig.show()

## Training Data Considerations 

Based on training data, graphs and tables, it is noted that most of the deaths were of people in 3rd class (Pclass), among the genders the most affected was men, 75% of the people who did not survive had below 37 years.

# Model

In [ ]:
# functions

def fit_and_predict(name, model, train_data, train_mark, test_data, test_mark):
    model.fit(train_data, train_mark)

    result = model.predict(test_data)

    hits = result == test_mark

    total_hits = sum(hits) 
    total_elements = len(test_data)

    hit_rate = 100.0 * total_hits / total_elements

    msg = "Algorithm Hit Rate {0}: {1}".format(name, hit_rate)
    
    print('classification result %.2f%%' %(model.score(test_data, test_mark)*100))
    print(msg)
    return hit_rate

def real_test(best, model, data_validation, mark_validation):
    result = model.predict(data_validation)
    hits = result == mark_validation

    total_hits = sum(hits)
    total_elements = len(mark_validation)

    hit_rate = 100.0 * total_hits / total_elements

    msg = "Winner hit rate {1} in the real world: {0}".format(hit_rate, best)
    print(msg)

def best_train_predict(name, model, train_data, train_mark, test_data_final, ID):
    model.fit(train_data, train_mark)

    predictions = model.predict(test_data_final)
    output = pd.DataFrame({'PassengerId': ID, 'Survived': predictions})
    output.to_csv('my_submission.csv', index=False)
    
    
    s = len(output[output.Survived==1])
    d = len(output[output.Survived==0])

    msg = "Algoritmo: {0} - S: {1} - D: {2}".format(name, s, d)

    print(msg)
    return output

Created 5 Classifiers and selected the best one

In [ ]:
features = ["Pclass", "Sex","Age" , "SibSp", "Parch", "Embarked", "Fare", "classes", 'Family_size']
X_df = train_data[features]
Y_df = train_data["Survived"]

Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

X = Xdummies_df.values
Y = Ydummies_df.values 

training_percentage = 0.3
test_percentage = 0.6

train_size = int(training_percentage * len(Y))
test_size = int(test_percentage * len(Y))
validation_size = len(Y) - train_size - test_size

train_data_x = X[:train_size]
train_mark_y = Y[:train_size]

test_end = train_size + test_size

test_data = X[train_size:test_end]
test_mark = Y[train_size:test_end]

data_validation = X[test_end:]
mark_validation = Y[test_end:]

# ------------------------------------------------------------------------------------------------------------------------------


from collections import Counter
from sklearn.naive_bayes import MultinomialNB
modelMultinomial = MultinomialNB()
resultMultinomial = fit_and_predict("MultinomialNB", modelMultinomial, train_data_x, train_mark_y, test_data, test_mark)

from sklearn.ensemble import AdaBoostClassifier
modelAdaBoost = AdaBoostClassifier(n_estimators=1000, random_state=1)
resultAdaBoost = fit_and_predict("AdaBoostClassifier", modelAdaBoost, train_data_x, train_mark_y, test_data, test_mark)

from sklearn.ensemble import RandomForestClassifier
modelRandomForestClassifier = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=1)
resulRandomForestClassifier = fit_and_predict("RandomForestClassifier", modelRandomForestClassifier, train_data_x, 
                                              train_mark_y, test_data, test_mark)

from sklearn.tree import DecisionTreeClassifier
modelDecisionTreeClassifier = DecisionTreeClassifier( max_depth=5, random_state=1)
resulDecisionTreeClassifier = fit_and_predict("DecisionTreeClassifier", modelDecisionTreeClassifier, train_data_x, 
                                              train_mark_y, test_data, test_mark)

from sklearn.ensemble import ExtraTreesClassifier
modelExtraTreesClassifier = ExtraTreesClassifier(n_estimators=1000, max_depth=5, random_state=1)
resulExtraTreesClassifier = fit_and_predict("ExtraTreesClassifier", modelExtraTreesClassifier, train_data_x, 
                                              train_mark_y, test_data, test_mark)


test_results_k = {'Multinomial':resultMultinomial, 
                'AdaBoost':resultAdaBoost, 
                'RandomForestClassifier':resulRandomForestClassifier, 
                'DecisionTreeClassifier':resulDecisionTreeClassifier, 
                'ExtraTreesClassifier':resulExtraTreesClassifier}

key_list = list(test_results_k.keys())
val_list = list(test_results_k.values())

position = val_list.index(max(val_list))

best = key_list[position]

if best == 'Multinomial':
    winner = modelMultinomial    
elif best == 'AdaBoost':
    winner = modelAdaBoost    
elif best == 'RandomForestClassifier':
    winner = modelRandomForestClassifier    
elif best == 'DecisionTreeClassifier':
    winner = modelDecisionTreeClassifier    
else:
    winner = modelExtraTreesClassifier  

real_test(best, winner, data_validation, mark_validation)

base_hit = max(Counter(mark_validation).values())
base_hit_rate = 100.0 * base_hit / len(mark_validation)
print("Base hit rate: %f" % base_hit_rate)

total_elements = len(data_validation)
print("Test total: %d" % total_elements)


X_predict = pd.get_dummies(test_data_final[features])
Best_result_modifi = best_train_predict(best, winner,X , Y, X_predict, test_data_final['PassengerId'])

Further more included two random variables that wasn't  correlated in any way with the target variable (survived):

 

*   random_num is a high cardinality numerical variable (as many unique values as records).
*   random_cat is a low cardinality categorical variable (3 possible values).

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

features = ["Pclass", "Sex","Age" , "SibSp", "Parch", "Embarked", "Fare", "classes", 'Family_size']
X_df = train_data[features]
Y_df = train_data["Survived"]

X = X_df
y = Y_df
rng = np.random.RandomState(seed=1)
X['random_cat'] = rng.randint(3, size=X.shape[0])
X['random_num'] = rng.randn(X.shape[0])

categorical_columns = ['Pclass', 'Sex', 'Embarked','classes','Family_size', 'random_cat']
numerical_columns = ['Age', 'SibSp', 'Parch', 'Fare', 'random_num']

X = X[categorical_columns + numerical_columns]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=1)

categorical_encoder = OneHotEncoder(handle_unknown='ignore')
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessing = ColumnTransformer(
    [('cat', categorical_encoder, categorical_columns),
     ('num', numerical_pipe, numerical_columns)])

rf = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', winner)
])
rf.fit(X_train, y_train)

In [ ]:
print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))

In [ ]:
ohe = (rf.named_steps['preprocess']
         .named_transformers_['cat'])
feature_names = ohe.get_feature_names(input_features=categorical_columns)
feature_names = np.r_[feature_names, numerical_columns]

tree_feature_importances = (
    rf.named_steps['classifier'].feature_importances_)
sorted_idx = tree_feature_importances.argsort()

y_ticks = np.arange(0, len(feature_names))
fig, ax = plt.subplots()
ax.barh(y_ticks, tree_feature_importances[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_yticklabels(feature_names[sorted_idx])
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
plt.show()
# Reference: https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py

This shows that the low cardinality categorical feature, sex and classes is the most important feature.

Also note that both random features have very low importances (close to 0) as expected.

In [ ]:
from sklearn.svm import SVC

from sklearn.metrics import plot_roc_curve


X = Xdummies_df
y = Ydummies_df

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
svc = SVC(random_state=1)
svc.fit(X_train, y_train)
# Next, we plot the ROC curve with a single call to 
# sklearn.metrics.plot_roc_curve. The returned svc_disp 
# object allows us to continue using the already computed 
# ROC curve for the SVC in future plots.
svc_disp = plot_roc_curve(svc, X_test, y_test)
plt.show()

# We train a random forest classifier and create a plot comparing it to the 
# SVC ROC curve. Notice how svc_disp uses plot to plot the SVC ROC curve 
# without recomputing the values of the roc curve itself. Furthermore, 
# we pass alpha=0.8 to the plot functions to adjust the alpha values of 
# the curves.
# Reference: https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_roc_curve_visualization_api.html#sphx-glr-auto-examples-miscellaneous-plot-roc-curve-visualization-api-py
rfc = winner
rfc.fit(X_train, y_train)
ax = plt.gca()
rfc_disp = plot_roc_curve(rfc, X_test, y_test, ax=ax, alpha=0.8)
svc_disp.plot(ax=ax, alpha=0.8)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import seaborn as sns
import matplotlib.pyplot as plt



train_data2 = train_data
test_data2 = test_data
treino_x, teste_x, treino_y, teste_y = train_data_x, test_data2, train_mark_y, test_mark

classificador = winner
classificador.fit(treino_x, treino_y)



selecionador_rfe = RFE(estimator = classificador, n_features_to_select = 9, step = 1)
selecionador_rfe.fit(treino_x, treino_y)
treino_rfe = selecionador_rfe.transform(treino_x)
teste_rfe = selecionador_rfe.transform(teste_x)
classificador.fit(treino_rfe, treino_y)

matriz_confusao = confusion_matrix(teste_y, classificador.predict(teste_rfe))
plt.figure(figsize = (10, 8))
sns.set(font_scale = 2)
sns.heatmap(matriz_confusao, annot = True, fmt = "d").set(xlabel = "Predição", ylabel = "Real")

print("Resultado da classificação %.2f%%" % (classificador.score(teste_rfe, teste_y)* 100))

In [ ]:
table_result_modifi = test_data_final
table_result_modifi['Survived'] = Best_result_modifi.Survived
table_result_modifi

In [ ]:
import plotly.graph_objects as go
Series_1 = table_result_modifi.query('Survived == 1')
Series_2 = table_result_modifi.query('Survived == 0')
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=Series_1['PassengerId'], 
                           y=Series_1['Age'], z=Series_1['Pclass'],name='Survived', 
                           mode='markers', 
                           marker=dict(size=5,
                           color='green',                # set color to an array/list of desired values
                           colorscale='Viridis',   # choose a colorscale
                           opacity=0.8),
                           text= 'agrupador'))

fig.add_trace(go.Scatter3d(x=Series_2['PassengerId'], 
                           y=Series_2['Age'], z=Series_2['Pclass'],name='Not Survived', 
                           mode='markers', 
                           marker=dict(size=5,
                           color='red',                # set color to an array/list of desired values
                           colorscale='Viridis',   # choose a colorscale
                           opacity=0.8),
                           text= 'agrupador'))

fig.update_layout(scene=dict(xaxis_title='PassengerId',
                             yaxis_title='Age', zaxis_title='Pclass'))

fig.update_layout(scene = dict(
                    zaxis = dict(
                        ticktext= ['1º - Class','2º - Class','3º - Class'],
                        tickvals= [1, 2, 3]))                    
                  )
# fig.update_layout(showlegend=True)

# fig.update_layout(legend_title_text = "Status")

fig.show()

References: 
* https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py

*  https://www.kaggle.com/gunesevitan/titanic-advanced-feature-engineering-tutorial/notebook